# Tests lab

In [3]:
import pyproj
from pyproj import  transformer, network
from pyproj.database import query_crs_info
import pandas as pd
import geopandas as gdp

network.set_network_enabled(True)


# List prebuilded crs

In [49]:
crs_list = list(query_crs_info(auth_name='EPSG'))
crs_list[0]._asdict()

{'auth_name': 'EPSG',
 'code': '2000',
 'name': 'Anguilla 1957 / British West Indies Grid',
 'type': <PJType.PROJECTED_CRS: 'PROJECTED_CRS'>,
 'deprecated': False,
 'area_of_use': AreaOfUse(west=-63.22, south=18.11, east=-62.92, north=18.33, name='Anguilla - onshore.'),
 'projection_method_name': 'Transverse Mercator'}

# How to use transformation

In [261]:
#always_xy garanty we do have lon lat always 
t:pyproj.Transformer = pyproj.transformer.TransformerGroup('5698', '9423', always_xy=True).transformers[0]

print(-120, 45, end="->")
print(t.transform(-114.12, 32.53))

print(t.area_of_use)
print(t.accuracy)
print(t.description)
print("-"*12)

# res:pyproj.CRS.coordinate_operation = 
# res.to_json_dict().get("id").get()

-120 45->(-1.363802263635756, -5.983693036872465)
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
-1.0
Inverse of Lambert-93 + Transformation from NGF-IGN69 height to EVRF2019 mean-tide height (ballpark vertical transformation) + Ballpark geographic offset from RGF93 v1 to ETRS89 + axis order change (2D)
------------


<Coordinate Operation: Other Coordinate Operation>
Name: Inverse of Lambert-93 + Transformation from NGF-IGN69 height to EVRF2019 mean-tide height (ballpark vertical transformation) + Ballpark geographic offset from RGF93 v1 to ETRS89
Method: PROJ-based operation method (approximate): +proj=pipeline +step +inv +proj=lcc +lat_0=46.5 +lon_0=3 +lat_1=49 +lat_2=44 +x_0=700000 +y_0=6600000 +ellps=GRS80 +step +proj=unitconvert +xy_in=rad +xy_out=deg +step +proj=axisswap +order=2,1
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)

In [117]:
# Transform a pandas table

dt = pd.DataFrame()
dt['x'] = [-120, -118.84841]
dt['y'] = [45, 39.48]
dt['z'] = [45, 35]


dt[['x', 'y', 'z']] = dt.apply(lambda row: pd.Series(t.transform(row['x'], row['y'], row['z'])), axis=1)
geometry = gdp.points_from_xy(dt['x'], dt['y'])
geometry

<GeometryArray>
[<POINT (6035628.635 5854817.168)>, <POINT (6321371.06 3820408.085)>]
Length: 2, dtype: geometry

# List of transformations
The main issue about listing transformations is the number; we have 6,400 coordinates in the reference system, which means we have approximately 6,400! transformations.

In [3]:
crs_list = list(query_crs_info(auth_name='EPSG'))
with open('file.txt', 'w') as file:
    for i in crs_list:
        for j in crs_list:
            try:
                for t in transformer.TransformerGroup(i.code, j.code).transformers:
                        file.write('\n'.join(t.description.split(" + ")))
                        file.write(str(t.transform_bounds)+'\n')
                        file.write(t.to_proj4()+'\n')
                        file.write('-----'*18+'\n')
                        file.write('-----'*18+'\n')
            except:
                pass

c:\Users\jakog\anaconda3\envs\pygdal\lib\site-packages\pyproj\transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=NLCSRSV4A.GSB, full_name=, package_name=, url=, direct_download=False, open_license=False, available=False)
  super().__init__(
c:\Users\jakog\anaconda3\envs\pygdal\lib\site-packages\pyproj\transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=ca_que_mern_cq77na83.tif, full_name=, package_name=, url=https://cdn.proj.org/ca_que_mern_cq77na83.tif, direct_download=True, open_license=True, available=True)
  super().__init__(
c:\Users\jakog\anaconda3\envs\pygdal\lib\site-packages\pyproj\transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=ca_que_mern_cq77na83.tif, full_name=https://cdn.proj.org/ca_que_mern_cq77na83.tif, package_name=, url=https://cdn.proj.org/ca_que_mern_cq77na83.tif, direct_download=True, open_license=True, available=Tr

# 3D transformation CRS building

In [5]:
from pyproj.crs import CompoundCRS, GeographicCRS, ProjectedCRS, VerticalCRS
from pyproj.crs.coordinate_system import Cartesian2DCS, VerticalCS
from pyproj.crs.coordinate_operation import LambertConformalConic2SPConversion

source_epsg = 4326
hieght_epsg= 5703
projected_epsg = 8718


vertcrs = pyproj.CRS.from_user_input(hieght_epsg)
projcrs = pyproj.CRS.from_user_input(projected_epsg)

compcrs = CompoundCRS(
    name='tmp', components=[projcrs, vertcrs]
)

t = transformer.TransformerGroup(source_epsg, projcrs, always_xy=True).transformers[0]
lat, long, z = 41.203323,  -77.194527, 17

t.transform(long,lat,  z)

CRSError: Invalid projection: {"$schema": "https://proj.org/schemas/v0.2/projjson.schema.json", "type": "CompoundCRS", "name": "tmp", "components": [{"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json", "type": "CompoundCRS", "name": "NAD83 / California zone 5 (ftUS) + NAVD88 height (ftUS)", "components": [{"type": "ProjectedCRS", "name": "NAD83 / California zone 5 (ftUS)", "base_crs": {"name": "NAD83", "datum": {"type": "GeodeticReferenceFrame", "name": "North American Datum 1983", "ellipsoid": {"name": "GRS 1980", "semi_major_axis": 6378137, "inverse_flattening": 298.257222101}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "id": {"authority": "EPSG", "code": 4269}}, "conversion": {"name": "SPCS83 California zone 5 (US Survey feet)", "method": {"name": "Lambert Conic Confor

In [3]:
import json

#load data
with open("./front/src/data/crs.json", "rb") as file:
    data = json.load(file)


unities = []
for item in data:
    unities.append(item.get('unity'))

unities = list(set(unities))
titles = unities.copy()

for i in range(len(unities)):
   unities[i] =  unities[i].replace('(', '')
   unities[i] =  unities[i].replace(')', '')
   unities[i] =  unities[i].replace(' ', '_')
   unities[i] =  unities[i].replace("'", '')
   unities[i] =  unities[i].upper()

for i, j in zip(unities, titles):
    print(f'{i} = "{j}", ')


KILOMETRE = "kilometre", 
US_SURVEY_FOOT = "US survey foot", 
BRITISH_YARD_SEARS_1922 = "British yard (Sears 1922)", 
METRE = "metre", 
CLARKES_FOOT = "Clarke's foot", 
GERMAN_LEGAL_METRE = "German legal metre", 
BRITISH_CHAIN_SEARS_1922_TRUNCATED = "British chain (Sears 1922 truncated)", 
BRITISH_FOOT_SEARS_1922 = "British foot (Sears 1922)", 
CLARKES_LINK = "Clarke's link", 
INDIAN_YARD = "Indian yard", 
LINK = "link", 
GRAD = "grad", 
BRITISH_CHAIN_SEARS_1922 = "British chain (Sears 1922)", 
BRITISH_FOOT_1936 = "British foot (1936)", 
DEGREE = "degree", 
GOLD_COAST_FOOT = "Gold Coast foot", 
FOOT = "foot", 
CLARKES_YARD = "Clarke's yard", 
